In [8]:
import os
import pandas as pd
import numpy as np
import json
import google.generativeai as genai
from servicios.pydantic_model import ProductDimensions
from dotenv import load_dotenv
from tavily import TavilyClient

In [ ]:
load_dotenv()
tavily_api_key = os.getenv("TAVILY_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")


'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'

In [4]:
df = pd.read_csv("files/test.csv")
df_llm = df.head(7).copy()

In [5]:
tavily_client = TavilyClient(api_key=tavily_api_key)
genai.configure(api_key=google_api_key)

In [ ]:
def extraer_datos_con_gemini(contexto: str, nombre_producto: str) -> dict:
    """Usa Gemini para extraer las dimensiones del texto de búsqueda."""
    model = genai.GenerativeModel('gemini-2.5-flash')
    
    prompt = f"""
    Basado en el siguiente contexto de búsqueda para el producto "{nombre_producto}", 
    extrae el alto, ancho, largo y peso.
    
    - Si encuentras las dimensiones exactas, úsalas.
    - Si no las encuentras, busca dimensiones de productos muy similares mencionados en el contexto.
    - Devuelve los valores como números flotantes (float).
    - Tu respuesta DEBE ser únicamente un objeto JSON con las claves "alto", "ancho", "largo", "peso".
    - En la clave "fuente" indica de que pagina web o fuente obtuviste la información.
    - Si no encuentras la fuente, pon "desconocida".
    - Si no puedes determinar alguno de los valores, intenta predecirlo basado en productos similares. determina que tipo de producto es y busca dimensiones típicas.
    - Selecciona la opción más representativa y confiable, utiliza las medidas que a veces suelen estar en el nombre de la publicación o en las imagenes. 
    - Sé especialmente ágil y eficiente al estimar medidas y **sobretodo** peso físico.

    Contexto:
    ---
    {contexto}
    ---
    """
    
    try:
        response = model.generate_content(prompt)
        # Limpiar la respuesta para asegurar que sea un JSON válido
        json_text = response.text.strip().replace("```json", "").replace("```", "")
        return json.loads(json_text)
    except (json.JSONDecodeError, Exception) as e:
        print(f"  - Error al procesar la respuesta de Gemini: {e}")
        return {}


def buscar_dimensiones_producto(nombre_producto: str) -> dict:
    """Usa Tavily para buscar y Gemini para extraer las dimensiones."""
    print(f"\n🔎 Buscando con Tavily: '{nombre_producto}'...")
    
    try:
        # Búsqueda avanzada con Tavily
        search_context = tavily_client.search(
            query=f'dimensions (height, width, length, weight) for product "{nombre_producto}"',
            search_depth="advanced" # Búsqueda más profunda
        )
        
        # Combinamos los resultados para dárselos a Gemini
        contexto_combinado = "\n".join([str(res) for res in search_context['results']])
        
        if not contexto_combinado:
            print("  - Tavily no devolvió resultados.")
            return {}
            
        print("  - Resultados de Tavily obtenidos. Extrayendo con Gemini...")
        datos_extraidos = extraer_datos_con_gemini(contexto_combinado, nombre_producto)
        
        if datos_extraidos:
            print("  - ✅ ¡Extracción con Gemini exitosa!")
            datos_extraidos['fuente'] = "Tavily + Gemini"
            return datos_extraidos
        else:
            print("  - 🟡 Gemini no pudo extraer los datos del contexto.")
            return {}

    except Exception as e:
        print(f"  - ❌ Error durante la búsqueda con Tavily: {e}")
        return {}

In [25]:
for name in df_llm['name']:
    print(f"\nProcesando producto: {name}")
    dimensiones = buscar_dimensiones_producto(name)
    productos = []
    if dimensiones:
        try:
            producto = ProductDimensions(**dimensiones)
            print(f"  - Dimensiones extraídas: {producto}")
            productos.append(producto)
        except Exception as e:
            print(f"  - Error al validar las dimensiones con Pydantic: {e}")
    else:
        print("  - No se pudieron extraer dimensiones para este producto.")


Procesando producto: SDR Rear Bumper Step footstep Trim For Toyota Fortuner 2016 onwards

🔎 Buscando con Tavily: 'SDR Rear Bumper Step footstep Trim For Toyota Fortuner 2016 onwards'...
  - Resultados de Tavily obtenidos. Extrayendo con Gemini...
  - ✅ ¡Extracción con Gemini exitosa!
  - Dimensiones extraídas: alto=7.5 ancho=20.0 largo=75.0 peso=0.5 fuente='Tavily + Gemini'

Procesando producto: ISEE 360® Car Bumper Sticker Exterior for Front Back Vinyl Checkered Flag Black Decals L X H 35 X 10 Cms

🔎 Buscando con Tavily: 'ISEE 360® Car Bumper Sticker Exterior for Front Back Vinyl Checkered Flag Black Decals L X H 35 X 10 Cms'...
  - Resultados de Tavily obtenidos. Extrayendo con Gemini...
  - ✅ ¡Extracción con Gemini exitosa!
  - Dimensiones extraídas: alto=10.0 ancho=0.1 largo=35.0 peso=250.0 fuente='Tavily + Gemini'

Procesando producto: AUTOWORLD MAX BOLERO FRONT GRILL WITH CROME CENTRE BLACK FOR MAHINDRA BOLERO TYPE-3

🔎 Buscando con Tavily: 'AUTOWORLD MAX BOLERO FRONT GRILL WITH

In [18]:
productos

[ProductDimensions(alto=8.0, ancho=6.0, largo=8.0, peso=140.0, fuente='Tavily + Gemini')]

In [ ]:
def extraer_datos_con_gemini(contexto: str, nombre_producto: str) -> dict:
    """Usa Gemini para extraer las dimensiones del texto de búsqueda."""
    model = genai.GenerativeModel('gemini-2.5-flash')
    
    prompt = f"""
    Basado en el siguiente contexto de búsqueda para el producto "{nombre_producto}", 
    Actuás como un analista especializado en pesos y medidas de productos. 
    Tu tarea consiste en analizar las medidas físicas de productos a partir de la información que te proporcionamos.
    Objetivo: Tu objetivo es estimar y devolver, para cada ítem: Alto (cm) Ancho (cm) Largo (cm) Peso (kg) 
    Podés ayudarte con publicaciones de sitios oficial o similares para inferir medidas realistas. 
    Selecciona la opción más representativa y confiable, utiliza las medidas que a veces suelen estar en el nombre de la publicación o en las imagenes. 
    Sé especialmente ágil y eficiente al estimar medidas y peso físico en los siguientes casos frecuentes: Usá la imagen si está disponible para inferir medidas.

    Contexto:
    ---
    {contexto}
    ---
    """
    
    try:
        response = model.generate_content(prompt)
        # Limpiar la respuesta para asegurar que sea un JSON válido
        json_text = response.text.strip().replace("```json", "").replace("```", "")
        return json.loads(json_text)
    except (json.JSONDecodeError, Exception) as e:
        print(f"  - Error al procesar la respuesta de Gemini: {e}")
        return {}